In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates. All rights reserved.

# Loading a pre-trained model in inference mode

In this tutorial, we will show how to instantiate a model pre-trained with VISSL to use it in inference mode to extract features from its trunk.

We will concentrate on loading a model pre-trained via SimCLR to use it in inference mode and extract features from an image, but this whole training is portable to any another pre-training methods (MoCo, SimSiam, SwAV, etc).

Through it, we will show:

1. How to instantiate a model associated to a pre-training configuration
2. Load the weights of the pre-trained model (taking the weights from our Model Zoo)
3. Use it to extract features associated to the VISSL Logo

**NOTE:** For a tutorial focused on how to use VISSL to schedule a feature extraction job, please refer to [the dedicated tutorial](https://colab.research.google.com/github/facebookresearch/vissl/blob/stable/tutorials/Feature_Extraction.ipynb)

**NOTE:** Please ensure your Collab Notebook has GPU available: `Edit -> Notebook Settings -> select GPU`.

**NOTE:** You can make a copy of this tutorial by `File -> Open in playground mode` and make changes there. DO NOT request access to this tutorial.


## Install VISSL

We will start this tutorial by installing VISSL, following the instructions [here](https://github.com/facebookresearch/vissl/blob/master/INSTALL.md#install-vissl-pip-package).

In [ ]:
# Install: PyTorch (we assume 1.5.1 but VISSL works with all PyTorch versions >=1.4)
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install opencv
!pip install opencv-python

# install apex by checking system settings: cuda version, pytorch version, python version
import sys
import torch
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{torch.__version__[0:5:2]}"
])
print(version_str)

# install apex (pre-compiled with optimizer C++ extensions and CUDA kernels)
!pip install -f https://dl.fbaipublicfiles.com/vissl/packaging/apexwheels/{version_str}/download.html apex

# install VISSL
!pip install vissl

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu101/torch-1.5.1%2Bcu101-cp37-cp37m-linux_x86_64.whl
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.5.1+cu101 which is incompatible.
ERROR: fairscale 0.3.2 has requirement torch>=1.6.0, but you'll have torch 1.5.1+cu101 which is incompatible.
  Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0


py37_cu102_pyt180
Looking in links: https://dl.fbaipublicfiles.com/vissl/packaging/apexwheels/py37_cu102_pyt180/download.html
  Using cached https://files.pythonhosted.org/packages/94/99/5861239a6e1ffe66e120f114a4d67e96e5c4b17c1a785dfc6ca6769585fc/torch-1.8.0-cp37-cp37m-manylinux1_x86_64.whl
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 1.8.0 which is incompatible.
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101


VISSL should be successfuly installed by now and all the dependencies should be available.

In [ ]:
import vissl
import tensorboard
import apex
import torch

## Loading a VISSL SimCLR pre-trained model


## Download the configuration

VISSL provides yaml configuration files for training a SimCLR model [here](https://github.com/facebookresearch/vissl/tree/master/configs/config/pretrain/simclr). We will start by fetching the configuration files we need.

In [ ]:
!mkdir -p configs/config/simclr
!mkdir -p vissl/config
!wget -q -O configs/config/simclr/simclr_8node_resnet.yaml https://raw.githubusercontent.com/facebookresearch/vissl/master/configs/config/pretrain/simclr/simclr_8node_resnet.yaml
!wget -q -O vissl/config/defaults.yaml https://raw.githubusercontent.com/facebookresearch/vissl/master/vissl/config/defaults.yaml



```
# This is formatted as code
```

## Download the ResNet-50 weights from the Model Zoo

In [ ]:
!wget -q -O resnet_simclr.torch https://dl.fbaipublicfiles.com/vissl/model_zoo/simclr_rn101_1000ep_simclr_8node_resnet_16_07_20.35063cea/model_final_checkpoint_phase999.torch

## Create the model associated to the configuration

Load the configuration and merge it with the default configuration.

In [ ]:
from omegaconf import OmegaConf
from vissl.utils.hydra_config import AttrDict

config = OmegaConf.load("configs/config/simclr/simclr_8node_resnet.yaml")
default_config = OmegaConf.load("vissl/config/defaults.yaml")
cfg = OmegaConf.merge(default_config, config)

Edit the configuration to freeze the trunk (inference mode) and ask for the extraction of the last layer feature.

In [ ]:
cfg = AttrDict(cfg)
cfg.config.MODEL.WEIGHTS_INIT.PARAMS_FILE = "resnet_simclr.torch"
cfg.config.MODEL.FEATURE_EVAL_SETTINGS.EVAL_MODE_ON = True
cfg.config.MODEL.FEATURE_EVAL_SETTINGS.FREEZE_TRUNK_ONLY = True
cfg.config.MODEL.FEATURE_EVAL_SETTINGS.EXTRACT_TRUNK_FEATURES_ONLY = True
cfg.config.MODEL.FEATURE_EVAL_SETTINGS.SHOULD_FLATTEN_FEATS = False
cfg.config.MODEL.FEATURE_EVAL_SETTINGS.LINEAR_EVAL_FEAT_POOL_OPS_MAP = [["res5avg", ["Identity", []]]]

And then build the model:

In [ ]:
from vissl.models import build_model

model = build_model(cfg.config.MODEL, cfg.config.OPTIMIZER)

## Loading the pre-trained weights

In [ ]:
from classy_vision.generic.util import load_checkpoint
from vissl.utils.checkpoint import init_model_from_weights

weights = load_checkpoint(checkpoint_path=cfg.config.MODEL.WEIGHTS_INIT.PARAMS_FILE)

init_model_from_weights(
    config=cfg.config,
    model=model,
    state_dict=weights,
    state_dict_key_name="classy_state_dict",
    skip_layers=[],  # Use this if you do not want to load all layers
)

print("Loaded...")

Loaded...


## Trying the model on the VISSL Logo

In [ ]:
!wget -q -O test_image.jpg https://raw.githubusercontent.com/facebookresearch/vissl/master/.github/logo/Logo_Color_Light_BG.png

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

image = Image.open("test_image.jpg")
image = image.convert("RGB")

pipeline = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])
x = pipeline(image)

features = model(x.unsqueeze(0))

The output is a list with as many representation layers as required in the configuration (in our case, `cfg.config.MODEL.FEATURE_EVAL_SETTINGS.LINEAR_EVAL_FEAT_POOL_OPS_MAP` asks for one representation layer, so we have just one output.

In [ ]:
features[0].shape

torch.Size([1, 2048, 1, 1])